In [1]:
import pandas as pd
from collections import OrderedDict
import pydicom
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image
import matplotlib.animation as animation
from IPython.display import HTML
import dotenv
import torch
import torchvision as tv
from torchvision.transforms import v2
from torchvision.models import resnet50, ResNet50_Weights
from torch.nn.functional import silu, sigmoid, softmax
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

%matplotlib notebook

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

## Data Loading

In [22]:
# the data root
root = '/kaggle/input/rsna-2023-abdominal-trauma-detection'

In [23]:
train_image_root = f'{root}/train_images'
train_data_path = f'{root}/train.csv'
scan_path = f'{root}/train_series_meta.csv'
injuries_path = f'{root}/image_level_labels.csv'
tag_path = f'{root}/train_dicom_tags.parquet'

In [24]:
labels = pd.read_csv(train_data_path)
scans = pd.read_csv(scan_path)
injuries = pd.read_csv(injuries_path)
tags = pd.read_parquet(tag_path)

In [25]:
class Config:
    TRAIN_SPLIT = 0.8
    LABELS = [
        "bowel_injury", "extravasation_injury",
        "kidney_healthy", "kidney_low", "kidney_high",
        "liver_healthy", "liver_low", "liver_high",
        "spleen_healthy", "spleen_low", "spleen_high",
    ]
    BATCH_SIZE = 128
    IMAGE_SIZE = 512
    NUM_EPOCHS = 10
    VAL_BATCH_SIZE=128
    BATCHES_PER_EVALUATION = 25
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
config = Config()

## Choosing the Training & Validation Images

In [26]:
label_by_patient = labels.groupby(['patient_id'])
def id_to_labels(row):
    return label_by_patient.get_group(row['patient_id']).squeeze()
# The injuries give us "interesting" images to use
data = pd.concat((injuries[['patient_id']].apply(id_to_labels, axis=1), injuries[['series_id', 'instance_number']]), axis=1)
data['image_path'] = train_image_root + '/' + data['patient_id'].astype(str) + '/' + data['series_id'].astype(str) + '/' + data['instance_number'].astype(str) + ".dcm"

In [7]:
data.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,image_path
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,362,/kaggle/input/rsna-2023-abdominal-trauma-detec...
1,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,363,/kaggle/input/rsna-2023-abdominal-trauma-detec...
2,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,364,/kaggle/input/rsna-2023-abdominal-trauma-detec...
3,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,365,/kaggle/input/rsna-2023-abdominal-trauma-detec...
4,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,366,/kaggle/input/rsna-2023-abdominal-trauma-detec...


In [8]:
# TRAIN_SPLIT of the patients will be used for training, the rest for validation
train_people = pd.Series(data['patient_id'].unique()).sample(frac=config.TRAIN_SPLIT)
train = data.loc[data['patient_id'].isin(train_people)]
val = data.loc[~data['patient_id'].isin(train_people)]

train.set_index(np.asarray(range(len(train))), inplace=True)
val.set_index(np.asarray(range(len(val))), inplace=True)

In [9]:
train.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,image_path
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,362,/kaggle/input/rsna-2023-abdominal-trauma-detec...
1,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,363,/kaggle/input/rsna-2023-abdominal-trauma-detec...
2,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,364,/kaggle/input/rsna-2023-abdominal-trauma-detec...
3,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,365,/kaggle/input/rsna-2023-abdominal-trauma-detec...
4,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,366,/kaggle/input/rsna-2023-abdominal-trauma-detec...


## Declare Dataset

In [10]:
image_augment = v2.Compose([
    v2.RandomResizedCrop(size=(config.IMAGE_SIZE, config.IMAGE_SIZE), scale=(0.8, 1.0), antialias=True),
])

In [27]:
def read_image(path):
    ds = pydicom.dcmread(path)
    return ds.pixel_array

class MedicalDataset(Dataset):
    def __init__(self, source: pd.DataFrame):
        self.image_paths = source['image_path']
        # 0/1 kidney injury, 0/1 fluid injury, kidney health (3 vals), liver health (3 vals), spleen health (3 vals)
        self.labels = torch.from_numpy(source[config.LABELS].to_numpy().astype('float32'))
        self.source = source
    def __len__(self):
        return len(self.source)
    def __getitem__(self, idx):
        image = torch.from_numpy(np.expand_dims(read_image(self.image_paths[idx]).astype('float32'), 0))
        image -= image.min()
        image /= image.max()
        
        image = torch.repeat_interleave(image, 3, dim=0)
        image = image_augment(image)
        # print(image.min(), image.max(), image.shape)
        if image.shape != (3, config.IMAGE_SIZE, config.IMAGE_SIZE):
            raise Exception(f'Image at {self.image_paths[idx]} has shape {image.shape}')
        label = (self.labels[idx][0:1], self.labels[idx][1:2], self.labels[idx][2:5], self.labels[idx][5:8], self.labels[idx][8:11])
        return image, label

In [28]:
train_ds = MedicalDataset(train)
val_ds = MedicalDataset(val)
train_dl = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=config.VAL_BATCH_SIZE)

## Declare Model

In [29]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.backbone = nn.Sequential(*(list(resnet50(weights=ResNet50_Weights.DEFAULT).children())[:-1]))
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.necks = nn.ModuleList([nn.Linear(2048, 32) for _ in range(5)])
        self.bowel = nn.Linear(32, 1)
        self.fluid = nn.Linear(32, 1)
        self.kidney = nn.Linear(32, 3)
        self.liver = nn.Linear(32, 3)
        self.spleen = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.flatten(self.backbone(x), start_dim=1)
        # bowel, fluid, kidney, liver, spleen
        necks = [silu(self.necks[i](x)) for i in range(5)]

        bowel = torch.flatten(sigmoid(self.bowel(necks[0])), start_dim=1)
        fluid = torch.flatten(sigmoid(self.fluid(necks[1])), start_dim=1)
        kidney = softmax(torch.flatten(self.kidney(necks[2]), start_dim=1), dim=1)
        liver = softmax(torch.flatten(self.liver(necks[3]), start_dim=1), dim=1)
        spleen = softmax(torch.flatten(self.spleen(necks[4]), start_dim=1), dim=1)
        return bowel, fluid, kidney, liver, spleen

In [14]:
classifier = Classifier()
classifier.to(device)
result = classifier(next(iter(train_dl))[0].to(device))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 76.7MB/s]


In [30]:
def calc_loss(output, target): # tuples 4D batched tensors please!
    bce = nn.BCELoss()
    cross = nn.CrossEntropyLoss()
    loss = bce(output[0], target[0]) + bce(output[1], target[1]) + cross(output[2], target[2]) + cross(output[3], target[3]) + cross(output[4], target[4])
    return loss
optimizer = torch.optim.Adam(classifier.parameters())

In [ ]:
def to_device(inputs, labels):
    return inputs.to(device), tuple(map(lambda i: i.to(device), labels))

for epoch in range(config.NUM_EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_dl):
        inputs, labels = to_device(*data)
        
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = calc_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        
        print(i, end="")
        running_loss += loss.item()
        if i % config.BATCHES_PER_EVALUATION == config.BATCHES_PER_EVALUATION-1:
            print()
            print(f'Epoch {epoch} batch {i} loss: {running_loss / config.BATCHES_PER_EVALUATION}')
            running_loss = 0.0
            
    total_val_loss = 0.0
    for data in val_dl:
        inputs, labels = to_device(*data)
        outputs = classifier(inputs)
        loss = calc_loss(outputs, labels)
        total_val_loss += loss.item()
    print(f'Epoch {epoch} validation loss: {total_val_loss / len(val_dl)}')

0123456789101112131415161718192021222324
Epoch 0 batch 24 loss: 3.2332051277160643
25262728293031323334353637383940414243444546474849
Epoch 0 batch 49 loss: 3.0786562538146973
50515253545556575859606162636465666768697071727374
Epoch 0 batch 74 loss: 2.9696616554260253
Epoch 0 validation loss: 3.5692181468009947
0123456789101112131415161718192021222324
Epoch 1 batch 24 loss: 2.833651885986328
25262728293031323334353637383940414243444546474849
Epoch 1 batch 49 loss: 2.705069761276245
505152535455565758596061626364